In [1]:
import os
from glob import glob
import dask
import xarray as xr
import cartopy.crs as ccrs
from cmocean import cm
import numpy as np
#from netCDF4 import Dataset
from scipy.io import loadmat

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)

w = cluster.start_workers(5)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.1.34:40109 Dashboard: http://10.148.1.34:8787/status,Cluster Workers: 10 Cores: 40 Memory: 500.00 GB


# Read rechunked data

In [9]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

ds_u = xr.open_zarr(out_dir_zarr+'U_total_rechunk.zarr')
ds_v = xr.open_zarr(out_dir_zarr+'V_total_rechunk.zarr')

In [10]:
ds_u_surface = ds_u.isel(depth=0)
ds_v_surface = ds_v.isel(depth=0)

In [17]:
ds_u_surface = ds_u_surface.chunk({'time': 9415, 'lat': 201, 'lon_g': 177})
ds_v_surface = ds_v_surface.chunk({'time': 9415, 'lat_g': 201, 'lon': 177})

In [18]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

file_out = out_dir_zarr+'U_surface_rechunk.nc'
%time ds_u_surface.to_netcdf(file_out, mode='w')     

file_out = out_dir_zarr+'V_surface_rechunk.nc'
%time ds_v_surface.to_netcdf(file_out, mode='w')   

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 216 ms, sys: 24 ms, total: 240 ms
Wall time: 3.06 s


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 148 ms, sys: 44 ms, total: 192 ms
Wall time: 2.76 s


# Surface KE

In [14]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

ds_u = xr.open_dataset(out_dir_zarr+'U_surface_rechunk.nc')
ds_v = xr.open_dataset(out_dir_zarr+'V_surface_rechunk.nc')
grid = xr.open_dataset(out_dir_zarr+'Grid.nc', decode_coords=True)

ds = xr.merge([ds_u,ds_v,grid])
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177, time: 9415)
Coordinates:
  * lat      (lat) float64 47.4 47.42 47.43 47.44 47.46 47.47 47.48 47.5 ...
  * lon_g    (lon_g) float64 -18.04 -18.02 -18.0 -17.98 -17.96 -17.94 -17.92 ...
    depth    float64 0.5
  * time     (time) float64 0.0 0.04167 0.08333 0.125 0.1667 0.2083 0.25 ...
  * lat_g    (lat_g) float64 47.4 47.41 47.42 47.44 47.45 47.46 47.48 47.49 ...
  * lon      (lon) float64 -18.03 -18.01 -17.99 -17.97 -17.95 -17.93 -17.91 ...
    XC       (lat, lon) float32 ...
    YC       (lat, lon) float32 ...
    XG       (lat_g, lon_g) float32 ...
    YG       (lat_g, lon_g) float32 ...
    CS       (lat, lon) float32 ...
    SN       (lat, lon) float32 ...
    Depth    (lat, lon) float32 ...
    dxC      (lat, lon_g) float32 ...
    dyC      (lat_g, lon) float32 ...
    dxG      (lat_g, lon) float32 ...
    dyG      (lat, lon_g) float32 ...
    rAz      (lat_g, lon_g) float32 ...
    rAw      (lat, lon_g) fl

In [7]:
import xrft
from dask.diagnostics import ProgressBar
import xgcm

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [16]:
gridx = xgcm.Grid(ds, coords={'X':{'center':'lon', 'left':'lon_g'},
                              'Y':{'center':'lat', 'left':'lat_g'}})
print(gridx)

<xgcm.Grid>
X Axis (periodic):
  * center   lon (177) --> left
  * left     lon_g (177) --> center
Y Axis (periodic):
  * center   lat (201) --> left
  * left     lat_g (201) --> center


In [38]:
KE = (gridx.interp(ds['U'].isel(time=slice(0,2160))**2,'X') + gridx.interp(ds['V'].isel(time=slice(0,2160))**2,'Y'))/2 # KE
KE

<xarray.DataArray (time: 2160, lat: 201, lon: 177)>
array([[[0.021097, 0.021399, ..., 0.032901, 0.020961],
        [0.026154, 0.026632, ..., 0.02927 , 0.018969],
        ...,
        [0.10715 , 0.099352, ..., 0.051726, 0.037394],
        [0.061756, 0.057555, ..., 0.042149, 0.029475]],

       [[0.02555 , 0.026171, ..., 0.030153, 0.01866 ],
        [0.032292, 0.033813, ..., 0.028187, 0.017164],
        ...,
        [0.079678, 0.077036, ..., 0.042795, 0.033252],
        [0.051071, 0.049693, ..., 0.034776, 0.02586 ]],

       ...,

       [[0.055764, 0.068383, ..., 0.027744, 0.016633],
        [0.058733, 0.066341, ..., 0.02865 , 0.014975],
        ...,
        [0.042911, 0.03527 , ..., 0.005918, 0.021083],
        [0.051794, 0.056295, ..., 0.008523, 0.015765]],

       [[0.059265, 0.067837, ..., 0.038223, 0.028779],
        [0.063936, 0.069696, ..., 0.03997 , 0.025034],
        ...,
        [0.049925, 0.043286, ..., 0.00554 , 0.021773],
        [0.056255, 0.057282, ..., 0.009274, 0.020503

In [39]:
out_dir_zarr = '/home1/datawork/xyu/OSMOSIS_llc4320/data_rechunk/'

file_out = out_dir_zarr+'KE_surface_rechunk.nc'
%time KE.to_netcdf(file_out, mode='w')     

CPU times: user 4 ms, sys: 92 ms, total: 96 ms
Wall time: 155 ms


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


In [41]:
cluster.scheduler.close()

<Future finished result=None>